# Comparison of sequential minimization strategies

Run the sequential minimization of the 10-parameter ansatz over the local and global cost functions with the fit method for the cost function sections.

In [ ]:
import sys
import logging
import numpy as np
import matplotlib.pyplot as plt

from qiskit import Aer, QuantumCircuit, transpile

logging.basicConfig(level=logging.WARNING, format='%(asctime)s: %(message)s')
logging.getLogger('schwinger_rqd').setLevel(logging.INFO)

sys.path.append('..')
from cost_sections import FitSecond, FitFirst, FitGeneral, FitSymmetric
from pnp_ansatze import make_pnp_ansatz
from cost_functions import global_cost_function, local_cost_function
from sequential_minimizer import SequentialVCMinimizer, SectionGenSwitcher, IdealCost
from rttgen import CNOTBasedRtt
from cx_decomposition import cx_circuit
from model_circuits import single_step, two_steps

In [ ]:
num_sites = 4
aJ = 1.
am = 0.5
omegadt = 0.2
# aJ = 0.7
# am = 0.439
# omegadt = 0.167
num_tstep = 6

backend = Aer.get_backend('statevector_simulator')
physical_qubits = None

In [ ]:
def make_compiler_circuit():
    qubit_pairs = list(zip(range(0, num_sites - 1), range(1, num_sites)))

    rtts = dict((qubits, CNOTBasedRtt(backend, qubits)) for qubits in qubit_pairs)
    cxs = dict((qubits, cx_circuit(backend, *qubits)) for qubits in qubit_pairs)
    cxs.update((qubits[::-1], cx_circuit(backend, *qubits[::-1])) for qubits in qubit_pairs)

    two_step_circuit = two_steps(num_sites, aJ, am, omegadt, rtts=rtts, cxs=cxs)

    target_circuit = QuantumCircuit(num_sites)
    target_circuit.x([0, 2])
    target_circuit.compose(two_step_circuit, inplace=True)

    uncomputer = make_pnp_ansatz(num_qubits=num_sites, num_layers=num_sites // 2, initial_x_positions=[1, 2], structure=[(1, 2), (0, 1), (2, 3)], first_layer_structure=[(0, 1), (2, 3)]).inverse()

    return target_circuit.compose(uncomputer)

In [ ]:
def make_minimizer(compiler_circuit, cost_function):
    section_generators = [FitGeneral] * len(compiler_circuit.parameters)
    if compiler_circuit.num_qubits == 2:
        section_generators[0] = FitSecond
        section_generators[1] = FitFirst
    elif compiler_circuit.num_qubits == 4:
        section_generators[0] = FitSecond
        section_generators[1] = FitFirst
        section_generators[2] = FitSecond
        section_generators[3] = FitFirst
        section_generators[5] = FitFirst

    minimizer = SequentialVCMinimizer(compiler_circuit, cost_function, section_generators, backend, shots_per_job=4096, num_jobs=1)

    return minimizer

In [ ]:
def callback_sweep(minimizer):
    state = minimizer.state
    if 'initial_param_val' in state:
        distance = np.max(np.abs(state['param_val'] - state['initial_param_val']))
        cost_update = state['cost'] - state['initial_cost']
    else:
        distance = 'n/a'
        cost_update = 'n/a'
        
    print('isweep', state['isweep'], 'update distance', distance, 'cost', state['cost'], 'cost update', cost_update, 'total shots', state['shots'])
    print('params [' + ', '.join(map(str, state['param_val'])) + ']')
    
def switch_to_gradient_descent(minimizer):
    print('Switching to gradient descent at cost', minimizer.state['cost'])
    minimizer.switch_strategy('gradient-descent')
        
def switch_to_gradient_descent_by_absolute_cost(minimizer):
    if minimizer.state['strategy'] != 'gradient-descent' and minimizer.state['cost'] < 0.02:
        switch_to_gradient_descent(minimizer)

def switch_to_gradient_descent_by_cost_update(minimizer):
    cost_update = minimizer.state['cost'] - minimizer.state['initial_cost']
    if minimizer.state['strategy'] != 'gradient-descent' and cost_update < 0. and cost_update > -0.1:
        switch_to_gradient_descent(minimizer)

In [ ]:
import multiprocessing

def run_minimizer(cost_type, strategy, queue):
    if cost_type == 'local':
        cost_function = local_cost_function
    else:
        cost_function = global_cost_function
        
    compiler_circuit = make_compiler_circuit()
    
    minimizer = make_minimizer(compiler_circuit, cost_function)
    
    ideal_cost = IdealCost(compiler_circuit)
    minimizer.callbacks_sweep.append(ideal_cost.callback_sweep)
        
    if '_gradient-descent' in strategy:
        minimizer.callbacks_sweep.append(switch_to_gradient_descent_by_cost_update)
    
    #switch_to_symmetric = SectionGenSwitcher(FitSymmetric, [4, 6, 7, 8, 9], 0.015)
    #minimizer.callbacks_sweep.append(switch_to_symmetric.callback_sweep)
    
    initial_param_val = np.ones(len(compiler_circuit.parameters), dtype='f8') * np.pi / 4.
    initial_strategy = strategy.replace('_gradient-descent', '')
    param_val = minimizer.minimize(initial_param_val, strategy=initial_strategy)
    
    if queue:
        queue.put((cost_type, strategy, param_val, ideal_cost.shots, ideal_cost.costs))
    else:
        return param_val, ideal_cost.shots, ideal_cost.costs
    
def driver():
    procs = dict()
    queue = multiprocessing.Queue()
    for cost_type in ['local', 'global']:
        for strategy in ['sequential', 'sequential_gradient-descent', 'largest-drop', 'largest-drop_gradient-descent']:
            proc = multiprocessing.Process(target=run_minimizer, args=(cost_type, strategy, queue))
            proc.start()
            procs[(cost_type, strategy)] = proc
            
    results = dict()
        
    while len(procs) != 0:
        cost_type, strategy, param_val, shots_values, cost_values = queue.get()
        print('{} {} returned'.format(cost_type, strategy))
        procs.pop((cost_type, strategy)).join()
        results[(cost_type, strategy)] = (param_val, shots_values, cost_values)
        
    return results    

In [ ]:
#results = driver()
results = dict()
for cost_type in ['local', 'global']:
    for strategy in ['sequential', 'sequential_gradient-descent', 'largest-drop', 'largest-drop_gradient-descent']:
        print(cost_type, strategy)
        results[(cost_type, strategy)] = run_minimizer(cost_type, strategy, None)

## Strategy comparison

In [ ]:
res = results[('local', 'sequential')]
plt.plot(res[1], res[2], label='local seq')
res = results[('local', 'largest-drop')]
plt.plot(res[1], res[2], label='local ld')
res = results[('local', 'sequential_gradient-descent')]
plt.plot(res[1], res[2], label='local seq_gd')
res = results[('local', 'largest-drop_gradient-descent')]
plt.plot(res[1], res[2], label='local ld_gd')
res = results[('global', 'sequential')]
plt.plot(res[1], res[2], label='global seq')
res = results[('global', 'largest-drop')]
plt.plot(res[1], res[2], label='global ld')
res = results[('global', 'sequential_gradient-descent')]
plt.plot(res[1], res[2], label='global seq_gd')
res = results[('global', 'largest-drop_gradient-descent')]
plt.plot(res[1], res[2], label='global ld_gd')
plt.yscale('log')
plt.legend()
plt.title('Two steps');

## Result validation

Largest-drop with local cost function converged the fastest for two Trotter steps. Are the resulting parameter values good enough? Check with RQD.

In [ ]:
qubit_pairs = list(zip(range(0, num_sites - 1), range(1, num_sites)))

rtts = dict((qubits, CNOTBasedRtt(backend, qubits)) for qubits in qubit_pairs)
cxs = dict((qubits, cx_circuit(backend, *qubits)) for qubits in qubit_pairs)
cxs.update((qubits[::-1], cx_circuit(backend, *qubits[::-1])) for qubits in qubit_pairs)
two_step_circuit = two_steps(num_sites, aJ, am, omegadt, rtts=rtts, cxs=cxs)

forward_circuit = make_pnp_ansatz(num_qubits=num_sites, num_layers=num_sites // 2, initial_x_positions=[1, 2], structure=[(1, 2), (0, 1), (2, 3)], first_layer_structure=[(0, 1), (2, 3)])
forward_circuit.compose(two_step_circuit, inplace=True)

param_dict = dict(zip(forward_circuit.parameters, results[('local', 'largest-drop')][0]))
approx_statevector = backend.run(transpile(forward_circuit.assign_parameters(param_dict), backend=backend)).result().results[0].data.statevector

true_circuit = QuantumCircuit(num_sites)
true_circuit.x([0, 2])
true_circuit.compose(two_step_circuit, inplace=True)
true_circuit.compose(two_step_circuit, inplace=True)

true_statevector = backend.run(transpile(true_circuit, backend=backend)).result().results[0].data.statevector

print(np.square(np.abs(approx_statevector.conjugate() @ true_statevector)))

In [ ]:
print(results[('local', 'largest-drop')][0])